In [ ]:
batch_size = 32
embedding_dims = 3647
hidden_dims = 250
epochs = 30
hidden_dims = 250

In [ ]:
import numpy as np
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten 
inp =  Input(shape=(embedding_dims, 1))
x = Conv1D(filters=1500, kernel_size=3, activation='relu',padding='valid', strides=1)(inp)
x = BatchNormalization(name='bn1_1')(x)
x = MaxPool1D(pool_size=3)(x)
x = Conv1D(filters=1000, kernel_size=3, activation='relu',padding='valid', strides=1)(x)
x = Dense(hidden_dims)(x)
x = (Dropout(0.5))(x)
x = BatchNormalization(name='bn1_2')(x)
pool = MaxPool1D(pool_size=3)(x)
flat = Flatten()(pool)
dense = Dense(1)(flat)
activation = (Activation('sigmoid'))(dense)
model = Model(inp, activation)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights1000.best.hdf5".format('boat_detector')
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=12) 
callbacks_list = [checkpoint, early, reduceLROnPlat]

history = model.fit(bows_spacy_train, y_train,batch_size=batch_size, epochs=epochs, validation_data=(bows_spacy_test, y_test),\
         callbacks=callbacks_list)